In [30]:
#IMPORTATION DE MODULES
#from textblob import TextBlob
import json
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from textblob import TextBlob




In [31]:
# Télécharger les stopwords en français si ce n'est pas déjà fait
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lnberroug/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lnberroug/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/lnberroug/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
#PARTIE : CREATION DU TWEET + SOUMISSION DU TWEET DANS LANDING ZONE

#Création d'une Classe Tweet à laquelle seront associées les attributs suivant : auteurs, hashtag, mentions, sentiments, (topics???)

class Tweet: #Plan/Modèle
    def __init__(self, auteur, contenu): #Méthode (assisstant qui aide à donner à chaque tweet ses attributs)
        self.auteur = auteur
        self.contenu = contenu

    def Nettoyage_Tweet(self):
        """Fonction qui supprime les caractères spéciaux d'un tweet"""
        enlever_speciaux = re.sub(r'[^a-zA-Z0-9\s@#]', '', self.contenu)
        return enlever_speciaux


def Zone_Atterisssage (Fichier): # incomplet
        """Stockage du tweet dans le fichier json zone d'atterissage"""
        
        ZA=[]
        with open (Fichier,'r',encoding='utf-8') as f:
            donnee= json.load(f)
            
        for infos_tweets in donnee:
            auteur = infos_tweets.get("author_id")
            contenu = infos_tweets.get("text")
            tweet_auteur = {
                "id": auteur,
                "tweet": Nettoyage_Tweet()
            }
            ZA.append(tweet_auteur)
        
        with open('ZA.json','w', encoding='utf-8') as f:
            json.dump(ZA, f, indent=4, ensure_ascii=False)

    
class ZA:
    def __init__(self,id , texte): #Méthode (assisstant qui aide à donner à chaque tweet ses attributs)
        self.id = id
        self.text = texte
        self.hashtags = self.Extraction_Hashtags()
        self.mentions = self.Extraction_Mentions()
        self.sentiment = self.Extraction_Sentiments()

    
    def Extraction_Sentiments(self): 
        """Extrait le sentiment du texte"""
        blob = TextBlob(self.text)
        polarite =  blob.sentiment.polarity
        if polarite > 0:
            sentiment = 'positif'
        elif polarite < 0:
            sentiment = 'négatif'
        elif polarite == 0:
            sentiment = 'neutre'
        return sentiment
    
    def Extraction_Hashtags(self):  #A Faire 
        """"""
        hashtags = re.findall(r'#\w+', self.text)
        return hashtags
    
    def Extraction_Mentions(self):  #A Faire
        """Fonction qui extrait les mentions de personnes dans un tweet"""
        mentions = re.findall(r'@[A-Za-z0-9_]+', self.text)

        return mentions


    def Id_Topic(tweet):

        def traitement_texte(text):
            text= text.lower() # mettre en minuscule
            text= re.sub(r'[^a-z\s]', '', text) 
            return text
        def get_stopwords():
            return stopwords.words('french')
        stopwords_list = get_stopwords()
        tweets= []
        with open ('ZA.json','r',encoding='utf-8') as f:
            donnee= json.load(f)
        
        tweets = [infos["tweet"] for infos in donnee]
        
        texte_traite = [traitement_texte(tweet) for tweet in tweets]
        
        vecteur = TfidfVectorizer(stop_words= stopwords_list)  # Utilise les stopwords en français intégrés dans sklearn
        a = vecteur.fit_transform(texte_traite)

        # n_components=2 : Le modèle cherchera à identifier 2 sujets principaux dans les données textuelles.
        lda = LatentDirichletAllocation(n_components=2, random_state=42)
        lda.fit(a) # on utilise LDA sur la matrice X 

        topics = []
        # Récupération du vocabulaire extrait par TfidfVectorizer
        words = vecteur.get_feature_names_out()
        for i, topic_weights in enumerate(lda.components_):
            top_indices = topic_weights.argsort()[:-3:-1]  # Prendre les 2 mots les plus importants
            top_words = [words[i] for i in top_indices]  # Utilisation des indices pour récupérer les mots correspondants dans la liste `words`.

            topics.append(f": {', '.join(top_words)}")

        tweet_traite = traitement_texte(tweet)
        vecteur = TfidfVectorizer(stop_words= stopwords_list) # Utilise les stopwords en français intégrés dans sklearn
        vecteur_tweet= vecteur.transform([tweet_traite]) 
        probabilities = lda.transform(vecteur_tweet)
        topic = probabilities.argmax()  # Trouve l'indice du topic avec la probabilité la plus élevée

        print(f"Le tweet :\n{tweet}\nAppartient au Topic {topics[topic]}")


        
# Process chaque tweet:
                
with open ('ZA.json','r') as file:
    tweets = json.load(file)


# --> il faut charger ces données dans une date frame : df = pd.DataFrame(tweets)





--> Pour améliorer ce code il faudrait utiliser nltk pour la tokenisation et la lemmenisation : ça va couper les tweet en petit mots et reduire les mots en leur forme conique ( je n'arrive pas à utilser nltk pour faire ça, erreur)

Ducoup la c pas précis

In [ ]:
t= "@ericbailly24 @maxigr04del  mes tontons vous avez fait votre part , JO prochain on ira en demi final au moins. BRAVO à vous . #SupportriceMazo #domie #CIV"


def Nettoyage_Tweet(tweet):
    """Fonction qui supprime les caractères spéciaux d'un tweet"""
    enlever_speciaux = re.sub(r'[^a-zA-Z0-9\s@#]', '', tweet)
    return enlever_speciaux

def Extraction_Sentiments(tweet): 
        """Extrait le sentiment du texte"""
        blob = TextBlob(tweet)
        polarite =  blob.sentiment.polarity
        if polarite > 0:
            sentiment = 'positif'
        elif polarite < 0:
            sentiment = 'négatif'
        elif polarite == 0:
            sentiment = 'neutre'
        return sentiment
    
def Extraction_Hashtags(tweet):  #A Faire 
        """"""
        hashtags = re.findall(r'#\w+', tweet)
        return hashtags
    
def Extraction_Mentions(tweet):  #A Faire
        """Fonction qui extrait les mentions de personnes dans un tweet"""
        mentions = re.findall(r'@[A-Za-z0-9_]+', tweet)
        return mentions


print(Nettoyage_Tweet(t))
print(Extraction_Sentiments(t))
print(Extraction_Hashtags(t))
print(Extraction_Mentions(t))




In [37]:
donne= [
    {"tweet": "Le réchauffement climatique menace la biodiversité."},
    {"tweet": "La pollution des océans est un problème mondial."},
    {"tweet": "Les voitures électriques sont une solution écologique."},
    {"tweet": "Les panneaux solaires deviennent plus efficaces chaque année."},
    {"tweet": "La déforestation accélère le changement climatique."}
]


def Id_Topic(tweet):

    def traitement_texte(text):
        text= text.lower() # mettre en minuscule
        text= re.sub(r'[^a-z\s]', '', text) 
        return text
    def get_stopwords():
        return stopwords.words('french')
    stopwords_list = get_stopwords()
    tweets= []
    
        
    tweets = [infos["tweet"] for infos in donne]
        
    texte_traite = [traitement_texte(tweet) for tweet in tweets]
        
    vecteur = TfidfVectorizer(stop_words= stopwords_list)  # Utilise les stopwords en français intégrés dans sklearn
    a = vecteur.fit_transform(texte_traite)

    # n_components=2 : Le modèle cherchera à identifier 2 sujets principaux dans les données textuelles.
    lda = LatentDirichletAllocation(n_components=2, random_state=42)
    lda.fit(a) # on utilise LDA sur la matrice X 

    topics = []
    # Récupération du vocabulaire extrait par TfidfVectorizer
    words = vecteur.get_feature_names_out()
    for i, topic_weights in enumerate(lda.components_):
        top_indices = topic_weights.argsort()[:-3:-1]  # Prendre les 2 mots les plus importants
        top_words = [words[i] for i in top_indices]  # Utilisation des indices pour récupérer les mots correspondants dans la liste `words`.

        topics.append(f": {', '.join(top_words)}")

    tweet_traite = traitement_texte(tweet)
    vecteur_tweet= vecteur.transform([tweet_traite]) 
    probabilities = lda.transform(vecteur_tweet)
    topic = probabilities.argmax()  # Trouve l'indice du topic avec la probabilité la plus élevée

    print(f"Le tweet :\n{tweet}\nAppartient au Topic {topics[topic]}")

tweet = "Le réchauffement climatique menace la biodiversité."
Id_Topic(tweet)


Le tweet :
Le réchauffement climatique menace la biodiversité.
Appartient au Topic : biodiversit, rchauffement
